In [1]:
import pandas as pd
from rich.console import Console
from rich.table import Table
from rich import box
import os
from datetime import datetime

#Updated to SEPTEMBER***

# Local file path on your machine, dynamically updated with the current date
file_path = f"C:\\Users\\jarrett.grose\\Desktop\\Invoicing\\pre invoice output 11.29.xlsx"

# Read the Excel file, assuming that the second row contains the correct headers
data = pd.read_excel(file_path, header=1)

# Alias mapping for the customers
alias_mapping = {
    #Stevenages
    "BOB": "BOB",
    "Name 1": "BOB",
    "Name 2": "BOB",
    "Name 3": "BOB",
    #Howdens
    "Name 1": "JOE",
    # ... any additional aliases
}

# Apply the alias mapping to the customer column
# Ensure that the mapping is applied correctly, considering the entire DataFrame or the specific 'Customer' column
data['Customer'] = data['Customer'].replace(alias_mapping, regex=True)

# Replace any customer name containing "STEVENAGE" with just "STEVENAGE" and "HOWDEN"
data['Customer'] = data['Customer'].str.replace(r'^.*BOB.*$', 'BOB', regex=True)
data['Customer'] = data['Customer'].str.replace(r'^.*JOE.*$', 'JOE', regex=True)

# Now 'customer_names' contains only the primary names, including "BOB"
customer_names = [
    "BLUE FERN", "HARDING", "PEARSON", "WILLOW",
    "BIRCHWOOD", "GOLDEN FIBRES", "STANFORDS", "ECO PACK", "SYNERGY", "VALENCIA",
    "PHOENIX", "CLOUD BOX", "QUANTUM", "ELMWOOD", "ASTRA", "FOXFORD", "HORIZON",
    # Note: No need to list the aliases here since they've been replaced in the data
]

# Correctly identify the columns by their names
order_no_column = 'Order No'
customer_column = 'Customer'  # Assuming it's the third column
dispatch_quantity_column = 'despatched qty'  # Assuming this is the correct column name
list_price_column = 'list price'
description_column = 'description'
long_description_column = 'long description'

# Round the 'list price' to the nearest whole number and convert to int to remove decimals
data[list_price_column] = data[list_price_column].round(0).astype(int)

# Trimming whitespace just in case
data[customer_column] = data[customer_column].str.strip()
data[description_column] = data[description_column].str.strip()
data[order_no_column] = data[order_no_column].astype(str).str.strip()

# Initialize 'Price Adjustment' column to an empty string
data['Price Adjustment'] = ''

# Specific customer names and their corresponding price adjustments
customer_based_price_adjustments = {
    'ECO PACK': '£2500',
    'WILLOW': '£1500',
    'VALENCIA': '£1750',
    'PHOENIX': '£300',
    'BLUE FERN': '£050',
    'CLOUD BOX': 'Check Price and Quantity with Sales',
    'ASTRA': 'Check With Sales',
    'FOXFORD': 'Proforma Invoice (Tell RL)',
    'SYNERGY LOGS': 'Proforma Invoice (Tell Manager)',
    'STELLAR CHEM': 'Proforma Invoice (Tell Manager)',
    'MAPLE VALLEY': 'Proforma Invoice (Tell Manager)',
    'VISTA CHOICE': 'Proforma Invoice (Tell Manager)',
    'FOXFORD': 'PAY ON INVOICE - CHECK THIS',
    # Add any additional specific price adjustments for other customers here
}

order_no_price_adjustments = {
    # MAPLE 
    '208209': '2100',
    # ASTRA 
    '208740': '1950',
    # Produce
    '209656': '£75.49 per unit',
    '210491': '£2190/ton',
}

# Dictionary of item codes, customers, and their corresponding price adjustments
item_customer_price_adjustments = {
    # Lenzing Orders
    ('0740X0000X220', 'GOLDEN FIBRES'): '£2600',
    ('1250X0000X220', 'GOLDEN FIBRES'): '£2600',
    ('1250X0000X120', 'GOLDEN FIBRES'): '£2650',
    # Magnet Orders
    ('0700X0000X025', 'SYNERGY'): '£2150',
    ('0430X0000X025', 'SYNERGY'): '£2150',
    ('0670X0000X075', 'SYNERGY'): '£2400',
    ('0670X0000X060', 'SYNERGY'): '£2230/£60.00 per reel',
    ('0600X0000X070', 'SYNERGY'): '£2230/£80.00 per reel',
    ('2000X0000X050', 'SYNERGY'): '£2200/£120.00 per reel',
    ('0250X0000X017', 'SYNERGY'): '£2550',
    ('0500X0000X020', 'SYNERGY'): '£2500',
    # JOE Orders
    ('0950X0000X050', 'JOE'): '£1900',
    ('1550X0000X050', 'JOE'): '£1900',
    ('1600X0000X050', 'JOE'): '£1900',
    ('0400X0000X030', 'JOE'): '£1950',
    ('0675X0000X030', 'JOE'): '£1950',
    ('0525X0000X080', 'JOE'): '£2050',
    ('0315X0000X040', 'JOE'): '£2300',
    ('0540X0000X050', 'JOE'): '£2300',
    ('0630X0000X060', 'JOE'): '£2300',
    ('0850X0000X050', 'JOE'): '£2300',
    # Pearson Orders
    ('0270X0000X038', 'PEARSON'): '£2150',
    ('0310X0000X030', 'PEARSON'): '£2200',
    ('0350X0000X030', 'PEARSON'): '£2200',
    ('0400X0000X038', 'PEARSON'): '£2150',
    ('0500X0000X017', 'PEARSON'): '£2250',
    # Elmwood Orders
    ('750X0000X012', 'ELMWOOD'): 'Ask Sales',
    ('800X0000X012', 'ELMWOOD'): 'Ask Sales',
    ('800X0000X013', 'ELMWOOD'): 'Ask Sales',
    ('850X0000X012', 'ELMWOOD'): 'Ask Sales',
    ('850X0000X013', 'ELMWOOD'): 'Ask Sales',
    ('850X0000X014', 'ELMWOOD'): 'Ask Sales',

    ('ITEM_CODE_2', 'CUSTOMER_2'): 'PRICE_2',
    # Add more item codes and customers with their prices here
}

# Additional rates for tints
additional_rates = {
    'BLUE': {15: 160, 30: 70, 50: 60, 60: 60, 80: 50},
    'RED': {15: 130, 30: 80, 50: 70, 60: 70, 80: 60},
    'GREEN': {15: 130, 30: 80, 50: 70, 60: 70, 80: 60}
}

# Function to adjust prices
def adjust_price(base_price, additional_rate):
    return base_price + additional_rate

# Dictionary of customers, keywords in long descriptions, and their corresponding price adjustments
customer_description_price_adjustments = {
    'HARDING': [
        ('GREEN', '£60.00 per unit'),
        ('VIOLET', '£60.20 per unit'),
        ('RED', '£60.10 per unit'),
        ('LT BLUE', '£55.00 per unit'),
        ('GREY', '£56.50 per unit'),
        ('BROWN', '£57.80 per unit'),
        ('YELLOW', '£60.05 per unit'),
        ('BLACK', '£58.10 per unit'),
        ('MINT GREEN', '£62.50 per unit'),
        ('PINK', '£62.80 per unit'),
        ('RIG 1DM - 30% PCR NATURAL MEDIUM SLIP WRAPPING FILM 2M X 60MU STRIP PERF', '£225.00 per unit')
    ],
    'JOE': [
        ('BLUE', '0950X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['BLUE'].get(50, 0))} per ton"),
        ('RED', '0950X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['RED'].get(50, 0))} per ton"),
        ('GREEN', '0950X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['GREEN'].get(50, 0))} per ton"),
        ('BLUE', '1550X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['BLUE'].get(50, 0))} per ton"),
        ('RED', '1550X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['RED'].get(50, 0))} per ton"),
        ('GREEN', '1550X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['GREEN'].get(50, 0))} per ton"),
        ('BLUE', '1600X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['BLUE'].get(50, 0))} per ton"),
        ('RED', '1600X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['RED'].get(50, 0))} per ton"),
        ('GREEN', '1600X0000X050', 'JOE', f"£{adjust_price(1950, additional_rates['GREEN'].get(50, 0))} per ton"),
        ('BLUE', '0400X0000X030', 'JOE', f"£{adjust_price(1955, additional_rates['BLUE'].get(30, 0))} per ton"),
        ('RED', '0400X0000X030', 'JOE', f"£{adjust_price(1955, additional_rates['RED'].get(30, 0))} per ton"),
        ('GREEN', '0400X0000X030', 'JOE', f"£{adjust_price(1955, additional_rates['GREEN'].get(30, 0))} per ton"),
    ],
    'ASTRA': [
        ('15235932', '£450.00'),
        ('15252789', '£260.00'),
        ('15235933', '£260.00'),
        ('15235931', '£250.00'),
    ]
}


# Apply item and customer-specific price adjustments first
for (item_code, customer_keyword), price_adjustment_value in item_customer_price_adjustments.items():
    condition = (
        data[description_column].str.contains(item_code, case=False, na=False) &
        data[customer_column].str.contains(customer_keyword, case=False, na=False)
    )
    data.loc[condition, 'Price Adjustment'] = price_adjustment_value

# Apply order number-based specific price adjustments
for order_no, new_price in order_no_price_adjustments.items():
    condition = (
        data[order_no_column].str.contains(order_no, case=False, na=False) &
        data['Price Adjustment'].eq('')  # Only update if no other adjustment is set
    )
    data.loc[condition, 'Price Adjustment'] = new_price    

    
# Apply customer-based specific price adjustments only where Price Adjustment is not already set
for customer, new_price in customer_based_price_adjustments.items():
    condition = (
        data[customer_column].str.contains(customer, case=False, na=False) &
        data['Price Adjustment'].eq('')
    )
    data.loc[condition, 'Price Adjustment'] = new_price

# Apply price adjustments based on customer name and keywords in long description
for customer, adjustments in customer_description_price_adjustments.items():
    for adjustment in adjustments:
        if len(adjustment) == 2:
            keyword, value = adjustment
            condition = (
                data[customer_column].str.contains(customer, case=False, na=False) &
                data[long_description_column].str.contains(keyword, case=False, na=False)
            )
            data.loc[condition, 'Price Adjustment'] = value
        elif len(adjustment) == 4:
            color, item_code, customer_keyword, value = adjustment
            condition = (
                data[customer_column].str.contains(customer_keyword, case=False, na=False) &
                data[description_column].str.contains(item_code, case=False, na=False) &
                data[long_description_column].str.contains(color, case=False, na=False)
            )
            data.loc[condition, 'Price Adjustment'] = value

# The "all else" condition for STEVENAGE
condition_steveange_all_else = (
    data[customer_column].str.contains('BOB', case=False, na=False) &
    (~data[long_description_column].str.contains('Laundry', case=False, na=False)) &
    (data['Price Adjustment'] == '')
)
data.loc[condition_steveange_all_else, 'Price Adjustment'] = 'Per System'

# Flag for 'Price Adjustment Needed' for customers in the customer_names list without a specific price
for customer in customer_names:
    condition = (
        data[customer_column].str.contains(customer, case=False, na=False) &
        data['Price Adjustment'].eq('')
    )
    data.loc[condition, 'Price Adjustment'] = 'Price Adjustment Needed'

# Plastic Tax Elsham/Over 15 tons quantity check
data['Remove Plastic Tax'] = data[customer_column].apply(lambda x: 'Remove Tax' if 'company name' in x.lower() else '')
data['Over Quantity'] = data[dispatch_quantity_column].apply(lambda x: 'Yes' if x > 30 else '')

# Specify columns to include in the final output
columns_to_display = [customer_column, order_no_column, dispatch_quantity_column, list_price_column, 'Price Adjustment', 'Over Quantity', 'Remove Plastic Tax']

# Selecting the final data for display
final_data = data[columns_to_display]

# Show all rows
pd.set_option('display.max_rows', None)

# Display the final data
final_data



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jarrett.grose\\Desktop\\Invoicing\\pre invoice output 11.29.xlsx'